### Acknowledgement
Name: Zuha Aqib     
ERP ID: 26106    
Section: 10am Miss Solat     
Date: (written on) 18-Feb-25     

code has been taken from Miss Solat's code files and written by Zuha themselves. 

In [177]:
from datetime import datetime
print("Last time code executed:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Last time code executed: 2025-02-23 20:22:26


In [178]:
erp = 26106 
# will be referenced later on in the code when we use it for random_state
erp

26106

# Imports
here we import all the necessary libraries and modules

In [179]:
# to load the dataset
import warnings
warnings.filterwarnings('ignore')

# to handle the data and perform numerical operations on it
import pandas as pd
import numpy as np

# to do natural language processing
import nltk

# preprocessing: to clean the data
import re

# preprocessing: words tokenization
from nltk.tokenize import word_tokenize

# preprocessing: stemmming and lemmatization
from nltk.stem import PorterStemmer, WordNetLemmatizer
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('punkt_tab')

# vectorization: BOW
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vectorization: LSA
from sklearn.decomposition import TruncatedSVD

# clustering: k means
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# to plot the graph
import matplotlib.pyplot as plt
import seaborn as sns

# Time Displayer
this code is a function that we will call at multiple instances of the code to see how long it took to run the code to see when it ran

In [180]:
# Function to get current date and time as a string
def get_current_datetime():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Load the Dataset
Here we load the dataset from a csv file and then save it into a variable

In [181]:
df = pd.read_csv("dataset/news_Feb_14.csv")
df = df.iloc[:, 0]  # Select only the headline column
documents = df.tolist()  # Convert to list
documents

['ECC endorses purchase of $582mn capital shares in BRICS’s New Development Bank',
 'Netanyahu’s statement to establish Palestinian state in Saudi Arabia irresponsible, provocative: FO',
 'India, US agree to resolve trade and tariff rows after Trump-Modi talks',
 'Aurangzeb discusses Pakistan’s structural reforms with IFC chief',
 'HBL, S&P Global launch Pakistan’s first manufacturing PMI',
 'KSE-100 closes 479 points lower as late-session selling erases intra-day gains',
 'PPL executes agreements for Sui Gas Field operations',
 'Hubco Green, PSO collaborate to setup EV charging infrastructure across Pakistan',
 'Rupee records marginal improvement against US dollar',
 'At least 9 dead, 7 injured in Balochistan’s Harnai blast',
 'Gold adds to record rally, reaches Rs306,200 per tola in Pakistan',
 'IMF’s 2024 Article IV report, and debate on inflation—I',
 'Pakistan CPI 2024 – deep-rooted corruption persists',
 '‘Like a wormhole’: Musk promises ‘Dubai Loop’ tunnel network',
 'Reforming 

In [182]:
original_headlines = documents.copy()  # Keep original headlines before preprocessing

# Preprocessing
here we perform the preprocessing of data like:
- lowercase text
- stopword removal
- stemming or lemmatization
- n-grams

## Lowercase Conversion
here we convert our text to lowercase to have accurate DTM/TDM

In [183]:
documents = [document.lower() for document in documents]

print("Finished executing at:", get_current_datetime())
documents

Finished executing at: 2025-02-23 20:22:26


['ecc endorses purchase of $582mn capital shares in brics’s new development bank',
 'netanyahu’s statement to establish palestinian state in saudi arabia irresponsible, provocative: fo',
 'india, us agree to resolve trade and tariff rows after trump-modi talks',
 'aurangzeb discusses pakistan’s structural reforms with ifc chief',
 'hbl, s&p global launch pakistan’s first manufacturing pmi',
 'kse-100 closes 479 points lower as late-session selling erases intra-day gains',
 'ppl executes agreements for sui gas field operations',
 'hubco green, pso collaborate to setup ev charging infrastructure across pakistan',
 'rupee records marginal improvement against us dollar',
 'at least 9 dead, 7 injured in balochistan’s harnai blast',
 'gold adds to record rally, reaches rs306,200 per tola in pakistan',
 'imf’s 2024 article iv report, and debate on inflation—i',
 'pakistan cpi 2024 – deep-rooted corruption persists',
 '‘like a wormhole’: musk promises ‘dubai loop’ tunnel network',
 'reforming 

## Data Cleaning
here we clean the data

In [184]:
import re
import unicodedata

def clean_text(text):
    text = text.encode('ascii', 'ignore').decode()  # Remove non-ASCII characters
    text = unicodedata.normalize("NFKD", text)  # Normalize Unicode text
    
    # Separate numbers attached to words
    text = re.sub(r'(?<=\d)(?=[a-zA-Z])', ' ', text)  # number-word
    text = re.sub(r'(?<=[a-zA-Z])(?=\d)', ' ', text)  # word-number
    
    text = text.replace("-", " ") # replace hyphens with spaces to tokenize the numbers and words
    text = re.sub(r'[^\w\s,]', '', text)  # Remove everything except words, numbers, and commas
    text = re.sub(r'\s+', ' ', text).strip()   # remove extra spaces
    
    return text

In [185]:
documents = [clean_text(text) for text in documents]
documents

['ecc endorses purchase of 582 mn capital shares in bricss new development bank',
 'netanyahus statement to establish palestinian state in saudi arabia irresponsible, provocative fo',
 'india, us agree to resolve trade and tariff rows after trump modi talks',
 'aurangzeb discusses pakistans structural reforms with ifc chief',
 'hbl, sp global launch pakistans first manufacturing pmi',
 'kse 100 closes 479 points lower as late session selling erases intra day gains',
 'ppl executes agreements for sui gas field operations',
 'hubco green, pso collaborate to setup ev charging infrastructure across pakistan',
 'rupee records marginal improvement against us dollar',
 'at least 9 dead, 7 injured in balochistans harnai blast',
 'gold adds to record rally, reaches rs 306,200 per tola in pakistan',
 'imfs 2024 article iv report, and debate on inflationi',
 'pakistan cpi 2024 deep rooted corruption persists',
 'like a wormhole musk promises dubai loop tunnel network',
 'reforming pension funds i

## Stop Words Removal
here we remove stop words. but we initialize stop words in count vectorizer. we have two other options of:
- either declaring all possible stop words and then iteratively checking each word in the document if its a stop word and then adding the non-stop words. However this is not a good practice. 
- pulling stop words from ```ntlk``` library:
```
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
documents_no_stopwords = [" ".join([word for word in doc.split() if word not in stop_words]) 
```
however after running this code it didnt not remove stop words thus i removed it

In [186]:
stop_word = False
stop_word_language = 'english'

stop_word

False

## Reduction
here we change words to a smaller common form instead of the 's. We do this using two ways:
- stemming
- lemmatization

### Stemming
this reduces words to their singular form however it is not said to be very good. we cannot say this surely as we have not rigourously tested it yet. 

In [187]:
# stemmer = PorterStemmer()
# documents = [" ".join([stemmer.stem(word) for word in doc.split()]) for doc in documents]

# print("Finished executing at:", get_current_datetime())
# documents

### Lemmatization
this also reduces words to their singular form and is said to be better as it only reduces to words in the dictionary. but again we cannot say this surely as we have not tested it yet. 

In [188]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_doc(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [189]:
documents = [lemmatize_doc(doc) for doc in documents]
documents

['ecc endorses purchase of 582 mn capital share in bricss new development bank',
 'netanyahus statement to establish palestinian state in saudi arabia irresponsible , provocative fo',
 'india , u agree to resolve trade and tariff row after trump modi talk',
 'aurangzeb discusses pakistan structural reform with ifc chief',
 'hbl , sp global launch pakistan first manufacturing pmi',
 'kse 100 close 479 point lower a late session selling erases intra day gain',
 'ppl executes agreement for sui gas field operation',
 'hubco green , pso collaborate to setup ev charging infrastructure across pakistan',
 'rupee record marginal improvement against u dollar',
 'at least 9 dead , 7 injured in balochistans harnai blast',
 'gold add to record rally , reach r 306,200 per tola in pakistan',
 'imf 2024 article iv report , and debate on inflationi',
 'pakistan cpi 2024 deep rooted corruption persists',
 'like a wormhole musk promise dubai loop tunnel network',
 'reforming pension fund in pakistan a su

## N-Grams
here we decide how we want the words to be tokenized, either they are
- unigram: singular
- bigram: pairs (doubular)

In [190]:
# n_gram_range = (1, 1) # unigram
n_gram_range = (2, 2) # bigram

n_gram_range

(2, 2)

# Vectorization
here we convert the text to numerical format for the model to understand it. we have four ways to do it:
- CountVectorizer using term-presence
- CountVextorizer using term-frequency
- TfidfVectorizer using term-frequency-inverse-document-frequency
- LSA using different 'n'

in this code we are exploring the last method LSA (TruncatedSVD)

## TF-IDF
here we construct a DTM (document to term matrix) using term-frequency inverse-data-frequency applied first before applying LSA

In [191]:
if stop_word is True:
    vectorizer = TfidfVectorizer(stop_words=stop_word_language, ngram_range=n_gram_range)
    documents = vectorizer.fit_transform(documents)
    print('Stop words removed')
else:
    vectorizer = TfidfVectorizer(ngram_range=n_gram_range)
    documents = vectorizer.fit_transform(documents)
    print('All words kept')

print("Finished executing at:", get_current_datetime())
documents.toarray()[:5]

All words kept
Finished executing at: 2025-02-23 20:22:27


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [192]:
# Get feature names (words) from CountVectorizer
feature_names = vectorizer.get_feature_names_out()

# Convert sparse matrix to DataFrame
df = pd.DataFrame(documents.toarray(), columns=feature_names)

df

,000 school,092 honda,10 bn,10 dead,10 movie,10 prize,10 th,10 turkey,10 year,100 close,...,zealand in,zealand provide,zelenskiy must,zelenskiy say,zelensky say,zeta jones,zia ijaz,zia mohyeddin,zone and,zoo will
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.316228,0.0,0.0,0.0
449,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
450,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
451,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [193]:
# Find (column, row) indices where frequency != 0
indices = np.argwhere(df.values != 0)

# Convert indices to a list of (word, row) pairs
word_occurrences = [(df.columns[col], row) for row, col in indices]

# Display the result
print(word_occurrences[:5])

[('582 mn', 0), ('bricss new', 0), ('capital share', 0), ('development bank', 0), ('ecc endorses', 0)]


## LSA
now we apply LSA, with different values of n (50, 100, 200). it is a way of reducing the matrix to lesser terms by making it more denser as our current matrix is more than 2000+ columns (regardless of stop words and ngrams)

In [194]:
n = 200
n

200

In [195]:
svd_model = TruncatedSVD(n_components=n, algorithm='randomized', n_iter=100, random_state=erp)
documents = svd_model.fit_transform(documents)
documents

array([[ 3.77508570e-17,  6.73970256e-17, -9.29743823e-17, ...,
         3.31779654e-02,  6.04217771e-02,  1.93096317e-02],
       [ 3.55525032e-04,  2.04201261e-08, -8.99759529e-17, ...,
        -2.47175430e-04,  2.67367644e-05, -7.90126520e-05],
       [ 4.06668954e-09,  9.96790923e-01,  6.50892069e-14, ...,
        -2.00071665e-06, -3.02827955e-07, -1.50913524e-05],
       ...,
       [-2.13316783e-18, -7.58882574e-19,  8.67445287e-18, ...,
        -2.88568251e-07, -3.67479293e-07,  4.76310305e-07],
       [-2.23985524e-18, -9.10439318e-18, -9.50359164e-17, ...,
        -4.29398714e-05, -3.21299434e-04, -9.11621249e-04],
       [ 1.16033608e-16,  1.16829164e-16, -1.18893719e-16, ...,
         1.32760447e-03,  8.89280976e-02,  4.87124153e-02]])

In [196]:
# Get feature names (words) from CountVectorizer
feature_names_lsa = svd_model.get_feature_names_out()

# Convert sparse matrix to DataFrame
df_lsa = pd.DataFrame(documents, columns=feature_names_lsa)

df_lsa

,truncatedsvd0,truncatedsvd1,truncatedsvd2,truncatedsvd3,truncatedsvd4,truncatedsvd5,truncatedsvd6,truncatedsvd7,truncatedsvd8,truncatedsvd9,...,truncatedsvd190,truncatedsvd191,truncatedsvd192,truncatedsvd193,truncatedsvd194,truncatedsvd195,truncatedsvd196,truncatedsvd197,truncatedsvd198,truncatedsvd199
0,3.775086e-17,6.739703e-17,-9.297438e-17,2.284702e-17,-5.465782e-17,1.544993e-18,9.444270e-17,-6.561335e-17,3.992136e-16,1.359157e-18,...,-5.409906e-02,1.910440e-02,-4.741450e-02,3.697609e-02,-9.151719e-04,2.967278e-02,5.846776e-02,3.317797e-02,6.042178e-02,1.930963e-02
1,3.555250e-04,2.042013e-08,-8.997595e-17,1.628930e-16,6.039818e-05,-9.539171e-18,-3.414918e-16,6.351908e-08,1.685211e-05,1.848131e-16,...,-8.867729e-07,-2.234967e-06,2.628150e-05,-5.623637e-06,-1.411414e-04,3.432668e-05,-6.304336e-05,-2.471754e-04,2.673676e-05,-7.901265e-05
2,4.066690e-09,9.967909e-01,6.508921e-14,5.254900e-14,-1.199094e-10,-1.938419e-16,1.292732e-16,-3.920520e-10,-9.102701e-08,7.012348e-17,...,1.050589e-06,-2.472931e-06,4.246492e-07,7.760991e-07,5.189205e-07,-3.182102e-06,-2.699337e-06,-2.000717e-06,-3.028280e-07,-1.509135e-05
3,1.824925e-17,-1.402153e-17,1.357366e-16,-6.469140e-17,-2.110495e-17,3.471789e-19,-1.575232e-16,6.646190e-17,-5.671271e-16,-4.262459e-18,...,-3.263800e-02,1.005910e-01,4.565809e-02,7.577310e-02,-5.922747e-02,1.184313e-01,-5.547930e-02,8.359506e-02,-1.755225e-01,3.429974e-02
4,-4.394631e-17,-2.053393e-17,-7.022079e-17,2.143323e-17,1.191085e-16,-3.419801e-18,-9.265034e-18,-1.808191e-17,-3.102424e-17,2.943928e-17,...,-7.969911e-17,-7.527931e-14,-9.136595e-14,-2.554184e-14,1.959275e-14,2.393451e-14,-7.878615e-15,4.200039e-14,1.954039e-14,-1.438854e-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,5.137890e-18,-3.652218e-17,1.165331e-17,-3.610092e-17,5.152790e-18,3.662627e-18,-1.743026e-16,7.452248e-17,-6.944134e-16,3.065349e-18,...,-1.208858e-01,7.225834e-02,4.344613e-02,5.343626e-02,7.185637e-02,5.016481e-02,-4.645025e-03,8.108623e-02,-1.289373e-01,-6.369296e-02
449,5.944730e-05,3.450800e-09,-1.411102e-16,9.470413e-18,1.025448e-05,2.823408e-18,2.543883e-18,1.103062e-08,2.925545e-06,-5.717396e-17,...,4.751543e-02,-2.275483e-02,7.254844e-02,1.368519e-02,-2.145444e-02,8.761910e-03,3.915321e-02,8.825049e-02,4.153116e-02,-7.481553e-03
450,-2.133168e-18,-7.588826e-19,8.674453e-18,-9.043044e-18,-2.058729e-17,7.239346e-18,4.217432e-19,-3.331198e-17,2.755772e-17,-5.397202e-17,...,2.130253e-07,3.439275e-08,4.314706e-07,3.971997e-08,-1.825105e-07,-1.176682e-08,-5.207691e-08,-2.885683e-07,-3.674793e-07,4.763103e-07
451,-2.239855e-18,-9.104393e-18,-9.503592e-17,-9.036043e-18,-1.520780e-17,1.413543e-17,1.321076e-03,1.402526e-15,4.653621e-16,-2.153756e-18,...,1.484055e-06,1.619012e-04,1.030220e-04,5.430243e-05,-1.336998e-05,7.224359e-06,-1.047425e-04,-4.293987e-05,-3.212994e-04,-9.116212e-04


# K Means Clustering

In [197]:
def display_k_means(k, data):
    print(f"Displaying {k} start time:", get_current_datetime())
    
    svd = TruncatedSVD(n_components=2, random_state=42)
    data = svd.fit_transform(data)
    
    kmeans = KMeans(n_clusters=k, random_state=erp)
    labels = kmeans.fit_predict(data)
    
    # Convert to DataFrame for visualization
    df_viz = pd.DataFrame({'X': data[:, 0], 'Y': data[:, 1], 'Cluster': labels})

    # Scatter plot of clusters
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_viz, x='X', y='Y', hue='Cluster', palette='tab10', s=100, edgecolor='black')
    plt.title("K-Means Clustering Visualization (2D Projection)")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.legend(title="Cluster")
    plt.show()
    
    print("Finished displaying at:", get_current_datetime(), "\n")

In [198]:
def run_k_means(k, data):
    print(f"Executing {k} start time:", get_current_datetime())
    
    kmeans = KMeans(n_clusters=k, random_state=erp)
    labels = kmeans.fit_predict(data)
    
    wss = kmeans.inertia_  # Within-Cluster Sum of Squares
    silhouette_avg = silhouette_score(data, labels)  # Silhouette Score
    
    print(f"K={k}: Silhouette Score and WSS={silhouette_avg:.4f}\t{wss:.4f}")
    
    values = [[silhouette_avg, wss]]
    # Create DataFrame and copy to clipboard
    df = pd.DataFrame(values)
    df.to_clipboard(index=False, header=False)
    print("copied to clipboard")
    
    # Create a DataFrame to store headlines with their assigned clusters
    df_clusters = pd.DataFrame({'Headline': original_headlines, 'Cluster': labels})

    # Display sample headlines per cluster
    for cluster in range(k):
        print(f"\nCluster {cluster}:")
        sample_headlines = df_clusters[df_clusters["Cluster"] == cluster].sample(n=min(5, len(df_clusters[df_clusters["Cluster"] == cluster])), random_state=erp)
        for idx, row in sample_headlines.iterrows():
            print(f"- {row['Headline']}")
    
    display_k_means(k, data)
    
    print("Finished executing at:", get_current_datetime(), "\n")

Executing 5 start time: 2025-02-23 20:22:47
K=5: Silhouette Score and WSS=0.0004	224.9484
copied to clipboard

Cluster 0:
- Justice Sarfraz Dogar sworn in as acting chief justice of IHC
- Justice Sarfraz Dogar takes oath as acting Chief Justice of IHC

Cluster 1:
- ECC endorses purchase of $582mn capital shares in BRICS’s New Development Bank
- Sindh minister calls for bold steps to modernise agriculture
- PM Modi Meets Elon Musk in Washington, Discusses Tech Ties Amid Trump Tariff Threats
- Volunteers’ climate taskforce to be set up for meaningful environmental action
- PM vows to advance objective of combating climate change

Cluster 2:
- PlayStation state of play returns with new games, trailers, and updates
- Hubco expands into EV charging sector with new subsidiary, partners with PSO
- Google partners with Poland to expand AI adoption in key sectors
- Supreme Court updates seniority list with new judges
- Hubco Green, PSO collaborate to setup EV charging infrastructure across Paki

Finished displaying at: 2025-02-23 20:22:47 

Finished executing at: 2025-02-23 20:22:47 



In [199]:
run_k_means(5, documents)

Executing 9 start time: 2025-02-23 20:22:47
K=9: Silhouette Score and WSS=-0.0198	221.3294
copied to clipboard

Cluster 0:
- Justice Sarfraz Dogar sworn in as acting chief justice of IHC
- Justice Sarfraz Dogar takes oath as acting Chief Justice of IHC

Cluster 1:
- Man gets four-year term in narcotics case
- Mariyam Nafees’ husband’s past marriage surprises fan
- JazzCash, Mobilink Bank named finalists for prestigious GLOMO Awards 2025
- Local bodies members flexing muscles for another protest in Peshawar
- Pakistan hopes hosting major cricket tournament will erase worries of instability

Cluster 2:
- Petrol Price in Pakistan to ‘go down’ from Feb 16 amid dip in global oil rates
- Reforming pension funds in Pakistan: a sustainable approach
- Harassment experiences in Pakistan: the need to speak up
- I want to become a UFC World Champion, Bano Butt on her dreams and breaking barriers
- Gold prices see another rise to reach record highs in Pakistan

Cluster 3:
- Man slays 4 kids before 

Finished displaying at: 2025-02-23 20:22:48 

Finished executing at: 2025-02-23 20:22:48 



In [200]:
run_k_means(9, documents)

Executing 13 start time: 2025-02-23 20:22:48
K=13: Silhouette Score and WSS=-0.0160	216.7375
copied to clipboard

Cluster 0:
- Justice Sarfraz Dogar sworn in as acting chief justice of IHC
- Justice Sarfraz Dogar takes oath as acting Chief Justice of IHC

Cluster 1:
- Non-banking microfinance sector: SECP announces series of initiatives to empower women, strengthen consumer protection
- If love hurts, the Sufis say you’re doing it right
- Senate panel advances nomination of Kash Patel as FBI director pick
- Andy Cohen’s alleged leaked nudes spark debate online
- Kinnaird College, PMAA equip young women with self-defense skills

Cluster 2:
- Petrol Price in Pakistan to ‘go down’ from Feb 16 amid dip in global oil rates
- Reforming pension funds in Pakistan: a sustainable approach
- Harassment experiences in Pakistan: the need to speak up
- I want to become a UFC World Champion, Bano Butt on her dreams and breaking barriers
- Gold prices see another rise to reach record highs in Pakistan

Finished displaying at: 2025-02-23 20:22:49 

Finished executing at: 2025-02-23 20:22:49 



In [201]:
run_k_means(13, documents)